# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [147]:
problem = '''Your goal is to invest in several of 10 possible investment strategies in the most optimal way. The historic returns of those strategies are stored in the file "investments_data.csv". Each column represents one strategy and the rows are the past investment outcomes. There is no index and the values are separated by a ;.

The costs for investing in a given investment is stored in a vector A, which has one value for each strategy in order.  
The values are: [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]

You can only invest once into an investment. 

Unfortunately due to other costs and inflation, your available budget at this time is uncertain. There are four possible budget scenarios with different probabilities: scenario 1 with 1000 euros and probability of 0.55, scenario 2  with 1100 euros and probability of 0.4, scenario 3 with 900 euros and probability of 0.04, scenario 4 with 1200 euros and probability of 0.01. 
The tolerable probability of exceeding the budget is 0.4.

Please formulate a mean-variance mathematical model for this optimization problem, considering the past performance of investment strategies and the uncertain budget. You can take 2 as the risk parameter r.'''

## 2. Generate the mathematical model

In [158]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [159]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Investment Strategy Selection

**Parameters:**

* **R:** Matrix of historical returns for each investment strategy (read from "investments_data.csv"). Each column represents a strategy, and each row represents a period.
* **A:** Vector of investment costs for each strategy: [80, 340, 410, 50, 180, 221, 15, 348, 191, 225].
* **S:** Vector of possible budget scenarios: [1000, 1100, 900, 1200].
* **P:** Vector of probabilities for each budget scenario: [0.55, 0.4, 0.04, 0.01].
* **r:** Risk parameter: 2.
* **α:** Tolerable probability of exceeding the budget: 0.4.

**Decision Variables:**

* **x:** Binary vector of length 10, where xᵢ = 1 if investment strategy i is selected, and xᵢ = 0 otherwise.

**Objective Function:**

Maximize the expected return while minimizing the portfolio variance:

```
Maximize:  ∑ᵢ (μᵢ * xᵢ) - r * ∑ᵢ∑ⱼ (σᵢⱼ * xᵢ * xⱼ)
```

where:

* μᵢ: Average historical return of investment strategy i (calculated from R).
* σᵢⱼ: Covariance between the returns of investment strategies i and j (calculated from R).

**Constraints:**

1. **Budget Constraint:**
   For each budget scenario k ∈ {1, 2, 3, 4}:
   ```
   ∑ᵢ (Aᵢ * xᵢ) ≤ Sₖ
   ```
2. **Probability of Budget Exceedance Constraint:**
   ```
   ∑ₖ (Pₖ * I(∑ᵢ (Aᵢ * xᵢ) > Sₖ)) ≤ α
   ```
   where I(condition) is an indicator function that equals 1 if the condition is true and 0 otherwise.
3. **Investment Uniqueness Constraint:**
   ```
   xᵢ ∈ {0, 1} for all i ∈ {1, 2, ..., 10}
   ```

**Model:**

The complete mathematical optimization model is:

```
Maximize:  ∑ᵢ (μᵢ * xᵢ) - r * ∑ᵢ∑ⱼ (σᵢⱼ * xᵢ * xⱼ)

Subject to:

∑ᵢ (Aᵢ * xᵢ) ≤ Sₖ for all k ∈ {1, 2, 3, 4}
∑ₖ (Pₖ * I(∑ᵢ (Aᵢ * xᵢ) > Sₖ)) ≤ α
xᵢ ∈ {0, 1} for all i ∈ {1, 2, ..., 10}
```

This model aims to find the optimal investment strategy selection (represented by the binary vector x) that maximizes the expected return while considering the risk (variance) and the uncertainty in the available budget. The budget constraint ensures that the total investment cost does not exceed the budget in each scenario. The probability of budget exceedance constraint limits the probability of exceeding the budget to the tolerable level α. The investment uniqueness constraint ensures that each investment strategy is selected at most once.


## 3. Generate the pyomo code

In [160]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [161]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo
import pandas as pd
import numpy as np

# Sample data - replace with actual data from "investments_data.csv"
data = {
    'Strategy 1': [0.02, 0.03, 0.01, 0.04],
    'Strategy 2': [0.05, 0.06, 0.04, 0.07],
    'Strategy 3': [0.03, 0.04, 0.02, 0.05],
    'Strategy 4': [0.07, 0.08, 0.06, 0.09],
    'Strategy 5': [0.01, 0.02, 0.0, 0.03],
    'Strategy 6': [0.04, 0.05, 0.03, 0.06],
    'Strategy 7': [0.06, 0.07, 0.05, 0.08],
    'Strategy 8': [0.02, 0.03, 0.01, 0.04],
    'Strategy 9': [0.08, 0.09, 0.07, 0.1],
    'Strategy 10': [0.05, 0.06, 0.04, 0.07]
}
df = pd.DataFrame(data)

R = df.values
A = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225] 
S = [1000, 1100, 900, 1200]
P = [0.55, 0.4, 0.04, 0.01]
r = 2
alpha = 0.4

# Calculate average returns and covariance matrix
mu = np.mean(R, axis=0)
Sigma = np.cov(R.T)

# Create Pyomo model
model = pyo.ConcreteModel()

# Define sets
model.I = pyo.RangeSet(0, len(A)-1)  # Set of investment strategies
model.K = pyo.RangeSet(0, len(S)-1)  # Set of budget scenarios

# Define decision variables
model.x = pyo.Var(model.I, within=pyo.Binary)

# Define objective function
def objective_rule(model):
  return sum(mu[i] * model.x[i] for i in model.I) - r * sum(Sigma[i, j] * model.x[i] * model.x[j] for i in model.I for j in model.I)
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Define constraints
model.budget_constraints = pyo.ConstraintList()
for k in model.K:
  model.budget_constraints.add(expr= sum(A[i] * model.x[i] for i in model.I) <= S[k])

model.prob_exceedance_constraint = pyo.Constraint(expr= sum(P[k] * int(sum(A[i] * model.x[i] for i in model.I) > S[k]) for k in model.K) <= alpha) 

# Solve the model
solver = pyo.SolverFactory('cbc')  # You can choose a different solver if needed
results = solver.solve(model)

# Print results
print(results)
print("Optimal investment strategy:")
for i in model.I:
  if pyo.value(model.x[i]) == 1:
    print(f"Invest in strategy {i+1}")
```

**Explanation:**

1. **Data Input:** 
    - The code starts by reading the historical returns from "investments_data.csv" (you need to replace the sample data with your actual data).
    - It defines the investment costs (A), budget scenarios (S), probabilities of scenarios (P), risk parameter (r), and tolerable probability of exceeding the budget (alpha).
2. **Data Preparation:**
    - The code calculates the average historical returns (mu) and the covariance matrix (Sigma) from the historical return data (R).
3. **Pyomo Model:**
    - A concrete model is created using `pyo.ConcreteModel()`.
    - Sets are defined for investment strategies (`model.I`) and budget scenarios (`model.K`).
    - Binary decision variables (`model.x`) are defined to indicate whether to invest in a strategy or not.
4. **Objective Function:**
    - The objective function (`model.objective`) is defined to maximize the expected return minus the risk (variance). The `objective_rule` function calculates this value.
5. **Constraints:**
    - `model.budget_constraints` ensures that the total investment cost does not exceed the budget in each scenario.
    - `model.prob_exceedance_constraint` limits the probability of exceeding the budget to the tolerable level alpha. This constraint uses an indicator function to check for budget exceedance in each scenario.
6. **Solving the Model:**
    - A solver (`cbc` in this case) is selected using `pyo.SolverFactory`.
    - The model is solved using `solver.solve(model)`.
7. **Printing Results:**
    - The solver results are printed.
    - The optimal investment strategy is printed by checking which decision variables (`model.x`) have a value of 1.

**Remember to replace the sample data with your actual data from "investments_data.csv".** 


## 4. Input problem data and try running the generated code

In [162]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np

# Sample data - replace with actual data from "investments_data.csv"
data = {
    'Strategy 1': [0.02, 0.03, 0.01, 0.04],
    'Strategy 2': [0.05, 0.06, 0.04, 0.07],
    'Strategy 3': [0.03, 0.04, 0.02, 0.05],
    'Strategy 4': [0.07, 0.08, 0.06, 0.09],
    'Strategy 5': [0.01, 0.02, 0.0, 0.03],
    'Strategy 6': [0.04, 0.05, 0.03, 0.06],
    'Strategy 7': [0.06, 0.07, 0.05, 0.08],
    'Strategy 8': [0.02, 0.03, 0.01, 0.04],
    'Strategy 9': [0.08, 0.09, 0.07, 0.1],
    'Strategy 10': [0.05, 0.06, 0.04, 0.07]
}
df = pd.DataFrame(data)

R = df.values
A = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225] 
S = [1000, 1100, 900, 1200]
P = [0.55, 0.4, 0.04, 0.01]
r = 2
alpha = 0.4

# Calculate average returns and covariance matrix
mu = np.mean(R, axis=0)
Sigma = np.cov(R.T)

# Create Pyomo model
model = pyo.ConcreteModel()

# Define sets
model.I = pyo.RangeSet(0, len(A)-1)  # Set of investment strategies
model.K = pyo.RangeSet(0, len(S)-1)  # Set of budget scenarios

# Define decision variables
model.x = pyo.Var(model.I, within=pyo.Binary)

# Define objective function
def objective_rule(model):
  return sum(mu[i] * model.x[i] for i in model.I) - r * sum(Sigma[i, j] * model.x[i] * model.x[j] for i in model.I for j in model.I)
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Define constraints
model.budget_constraints = pyo.ConstraintList()
for k in model.K:
  model.budget_constraints.add(expr= sum(A[i] * model.x[i] for i in model.I) <= S[k])

model.prob_exceedance_constraint = pyo.Constraint(expr= sum(P[k] * int(sum(A[i] * model.x[i] for i in model.I) > S[k]) for k in model.K) <= alpha) 

# Solve the model
solver = pyo.SolverFactory('cbc')  # You can choose a different solver if needed
results = solver.solve(model)

# Print results
print(results)
print("Optimal investment strategy:")
for i in model.I:
  if pyo.value(model.x[i]) == 1:
    print(f"Invest in strategy {i+1}")

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'InequalityExpression'

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.

In [163]:
print(response.text)

## Mathematical Optimization Model for Investment Strategy Selection

**Parameters:**

* **R:** Matrix of historical returns for each investment strategy (read from "investments_data.csv"). Each column represents a strategy, and each row represents a period.
* **A:** Vector of investment costs for each strategy: [80, 340, 410, 50, 180, 221, 15, 348, 191, 225].
* **S:** Vector of possible budget scenarios: [1000, 1100, 900, 1200].
* **P:** Vector of probabilities for each budget scenario: [0.55, 0.4, 0.04, 0.01].
* **r:** Risk parameter: 2.
* **α:** Tolerable probability of exceeding the budget: 0.4.

**Decision Variables:**

* **x:** Binary vector of length 10, where xᵢ = 1 if investment strategy i is selected, and xᵢ = 0 otherwise.

**Objective Function:**

Maximize the expected return while minimizing the portfolio variance:

```
Maximize:  ∑ᵢ (μᵢ * xᵢ) - r * ∑ᵢ∑ⱼ (σᵢⱼ * xᵢ * xⱼ)
```

where:

* μᵢ: Average historical return of investment strategy i (calculated from R).
* σᵢⱼ: Covaria

In [164]:
print(response2.text)

```python
import pyomo.environ as pyo
import pandas as pd
import numpy as np

# Sample data - replace with actual data from "investments_data.csv"
data = {
    'Strategy 1': [0.02, 0.03, 0.01, 0.04],
    'Strategy 2': [0.05, 0.06, 0.04, 0.07],
    'Strategy 3': [0.03, 0.04, 0.02, 0.05],
    'Strategy 4': [0.07, 0.08, 0.06, 0.09],
    'Strategy 5': [0.01, 0.02, 0.0, 0.03],
    'Strategy 6': [0.04, 0.05, 0.03, 0.06],
    'Strategy 7': [0.06, 0.07, 0.05, 0.08],
    'Strategy 8': [0.02, 0.03, 0.01, 0.04],
    'Strategy 9': [0.08, 0.09, 0.07, 0.1],
    'Strategy 10': [0.05, 0.06, 0.04, 0.07]
}
df = pd.DataFrame(data)

R = df.values
A = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225] 
S = [1000, 1100, 900, 1200]
P = [0.55, 0.4, 0.04, 0.01]
r = 2
alpha = 0.4

# Calculate average returns and covariance matrix
mu = np.mean(R, axis=0)
Sigma = np.cov(R.T)

# Create Pyomo model
model = pyo.ConcreteModel()

# Define sets
model.I = pyo.RangeSet(0, len(A)-1)  # Set of investment strategies
model.K = 